In [1]:
from bs4 import BeautifulSoup, NavigableString
import re

In [2]:
with open('wfg1.html', 'r') as f:
    html = ''.join(f.readlines())

In [3]:
soup = BeautifulSoup(html, 'lxml')

In [4]:
# largest element
main = max([*soup.find_all('blockquote')], key=len)
print(main.prettify())

<blockquote>
 <i>
 </i>
 <dd>
  <i>
   Estragon, sitting on a low mound, is trying to take off his boot. He pulls at it with both hands, panting.
   <font color="#ffffff" size="0">
    #
   </font>
  </i>
  <blockquote>
   <i>
   </i>
   <dd>
    <i>
     <img src="tft2/godot/prodphot/07.jpg"/>
     <br/>
    </i>
    <font face="arial" size="1">
     Image from timil.com
    </font>
    <i>
    </i>
   </dd>
  </blockquote>
  <i>
  </i>
 </dd>
 <dd>
  <i>
   He gives up, exhausted, rests, tries again.
  </i>
 </dd>
 <dd>
  <i>
   As before.
  </i>
 </dd>
 <dd>
  <i>
   Enter Vladimir.
  </i>
 </dd>
 <dt>
  <b>
   ESTRAGON:
  </b>
 </dt>
 <dd>
  (
  <i>
   giving up again
  </i>
  )
  <i>
   .
  </i>
  Nothing to be done.
 </dd>
 <dt>
  <b>
   VLADIMIR:
  </b>
 </dt>
 <dd>
  (
  <i>
   advancing with short, stiff strides, legs wide apart
  </i>
  )
  <i>
   .
  </i>
  I'm beginning to come round to that opinion.  All my life I've tried to put it from me, saying Vladimir, be reasonable,

In [5]:
with open('wfg1.txt', 'w', encoding='utf-8') as f:
    current_text = ''
    current_speaker = ''
    for tag in main:
        if tag.name == 'dt':
            current_text = ''
            current_speaker = tag.b.text

            # italic is not conversation
            temp_tag = tag.find_next_sibling('dd', recursive=False)
            while temp_tag and temp_tag.name == 'dd':
                if not temp_tag.find_all('div') and not temp_tag.find_all('font', {'face': 'arial'}):
                    for inner_tag in temp_tag:
                        if type(inner_tag) is NavigableString:
                            current_text += inner_tag
                        else:
                            if inner_tag.name != 'i' and inner_tag.name != 'blockquote' and not inner_tag.find_all('i') and not inner_tag.find_all('blockquote'):
                                current_text += inner_tag.text
                temp_tag = temp_tag.next_sibling

            if tag.find('img'):
                # directly creat an entry
                current_text = ''.join([inner_tag if type(inner_tag) is NavigableString else inner_tag.text for inner_tag in tag if inner_tag.name != 'b'])

            current_text = current_text.replace('\n', '').replace('. . .', '…').replace('#', '')
            current_text = re.sub(r'\((.*?)\)', '', current_text)
            current_text = re.sub(r' +', ' ', current_text)
            current_text = current_text.replace('… …', '…').strip()
            if current_text:
                f.write(current_speaker + ' ' + current_text + '\n')